In [1]:
import torch
import pandas as pd
import numpy as np
import datasets
from load_models_and_data import load_vocabulary, load_embeddings, text_to_embeddings, calc_cosine_sim, calculate_embeddings, create_packed_batch
from tqdm import tqdm
tqdm.pandas()
#from TwoTowerNN import QryTower, DocTower, TripletEmbeddingDataset, run_hyperparameter_tuning
from TwinTowerGRU import QryTower, DocTower, EmbeddingTripletDataset, run_hyperparameter_tuning, GRUTwinTowerModel
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader,  SubsetRandomSampler
from sklearn.model_selection import KFold, train_test_split
import os
import wandb
from dotenv import load_dotenv
import torch.nn.functional as F

/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


API key loaded successfully


wandb: Currently logged in as: nnamdi-odozi (nnamdi-odozi-ave-actuaries) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from datasets import load_dataset

# Loading datasets from Hugging Face
ds_soft_neg = load_dataset("cocoritzy/week_2_triplet_dataset_soft_negatives")
#ds_hard_neg = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")


In [3]:
# Paths to your files
embeddings_path = "./downloaded_model/glove_embeddings.pt" #set this to either own-trained cbow ones or to glove pre-trained ones
vocab_path = "./downloaded_model/glove_ids_to_words.csv"

# Load embeddings and vocabulary
print("Loading embeddings and vocabulary...")
embeddings = load_embeddings(embeddings_path)
word_to_idx = load_vocabulary(vocab_path)

print(f"Loaded embeddings with shape: {embeddings.shape}")
print(f"Loaded vocabulary with {len(word_to_idx)} tokens")

# Example usage (uncomment when ready to test)
sample_text = "This is a test sentence"
embeddings_result, length = text_to_embeddings(sample_text, word_to_idx, embeddings, is_query=True)
print(f"Embedded text shape: {embeddings_result.shape}")

# Testing - Set numpy print options
np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)
print("Length is:", length)


Loading embeddings and vocabulary...
Loaded embeddings with shape: torch.Size([400000, 100])
Loaded vocabulary with 399998 tokens
Embedded text shape: torch.Size([26, 100])
Embedding array with custom formatting:
[[ 0.2616  0.4472 -0.0968 ... -0.4503  0.4952 -0.203 ]
 [ 0.1372 -0.5429  0.1942 ... -0.5206  0.2543 -0.2376]
 [-0.3046 -0.2365  0.1758 ... -0.8456 -0.0354  0.1704]
 ...
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]]
Length is: 5


In [ ]:
sample_text = ""
embeddings_result, length = text_to_embeddings(sample_text, word_to_idx, embeddings, is_query=True)
print(f"Embedded text shape: {embeddings_result.shape}")

np.set_printoptions(precision=4, suppress=True, threshold=10)  # threshold limits number of elements shown
numpy_array = embeddings_result.detach().numpy()
print("Embedding array with custom formatting:")
print(numpy_array)
print("Length is:", length)


In [ ]:
df_soft_neg  = pd.DataFrame(ds_soft_neg['train'])
#df_hard_neg  = pd.DataFrame(ds_hard_neg['train'])

In [ ]:
embedded_query, length = text_to_embeddings(df_soft_neg['query'][0], word_to_idx, embeddings, is_query=True)
embedded_positive, length = text_to_embeddings(df_soft_neg['positive_passage'][0], word_to_idx, embeddings, is_query=False)
embedded_negative, length = text_to_embeddings(df_soft_neg['negative_passage'][0], word_to_idx, embeddings, is_query=False)

print(embedded_positive.shape)
print(embedded_negative.shape)

In [ ]:
a = embedded_query.mean(dim=0)
b = embedded_positive.mean(dim=0)
c = embedded_negative.mean(dim=0)
a.shape


In [ ]:
import torch.nn.functional as F

cosine_similarity = F.cosine_similarity(a, c, dim=0)
print(f"Cosine similarity between query and positive passage: {cosine_similarity.item()}")

In [ ]:

# # Process the dataframe using apply just for first five rows
# print("Calculating similarities... This may take a while depending on dataframe size.")
# similarities = df_soft_neg[0:5].progress_apply(
#     lambda row: calculate_similarities(row, word_to_idx, embeddings), 
#     axis=1
# )

# # Join the similarities to the dataframe
# df_soft_neg_ext = pd.concat([df_soft_neg[0:5], similarities], axis=1)

# # Show a sample of the results
# #print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())
#print(df_soft_neg_ext.head())
#print(df_soft_neg_ext.columns)

In [ ]:

# Process the dataframe using apply
print("Calculating embeddings... This may take a while depending on dataframe size.")
embeddings_padded = df_soft_neg.progress_apply(
    lambda row: calculate_embeddings(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_soft_neg_ext = pd.concat([df_soft_neg, embeddings_padded], axis=1)
print(df_soft_neg_ext.head())
# Show a sample of the results
#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_soft_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_soft_neg_ext['query_pos_sim'] > df_soft_neg_ext['query_neg_sim']).sum()
#total = len(df_soft_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
df_soft_neg_ext[0:1]

In [ ]:
# Process the dataframe using apply
print("Calculating embeddings... This may take a while depending on dataframe size.")
embeddings_padded = df_hard_neg.progress_apply(
    lambda row: calculate_embeddings(row, word_to_idx, embeddings), 
    axis=1
)

# Join the similarities to the dataframe
df_hard_neg_ext = pd.concat([df_hard_neg, embeddings_padded], axis=1)
print(df_hard_neg_ext.head())
# Show a sample of the results
#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].head())

#print(df_hard_neg_ext[['query_pos_sim', 'query_neg_sim', 'pos_neg_sim']].mean())

# Calculate how often the positive passage is ranked higher than negative
#higher_count = (df_hard_neg_ext['query_pos_sim'] > df_hard_neg_ext['query_neg_sim']).sum()
#total = len(df_hard_neg_ext)
#print(f"\nPositive passage ranked higher than negative: {higher_count} out of {total} ({higher_count/total:.2%})")



In [ ]:
df_all_neg_ext = pd.concat([df_soft_neg_ext, df_hard_neg_ext])
df_all_neg_ext.head()

In [ ]:
# Save DataFrames to pickle format
df_soft_neg_ext.to_pickle("downloaded_model/df_soft_neg_ext.pkl")
#df_hard_neg_ext.to_pickle("downloaded_model/df_hard_neg_ext.pkl")
#df_all_neg_ext.to_pickle("downloaded_model/df_all_neg_ext.pkl")

In [4]:
# Function to load a DataFrame from pickle if the file exists
def load_df_if_exists(file_path):
    if os.path.exists(file_path):
        return pd.read_pickle(file_path)
    else:
        print(f"File not found: {file_path}")
        return None

# Load DataFrames
df_soft_neg_ext = load_df_if_exists("downloaded_model/df_soft_neg_ext.pkl")
#df_hard_neg_ext = load_df_if_exists("downloaded_model/df_hard_neg_ext.pkl")
#df_all_neg_ext = load_df_if_exists("downloaded_model/df_all_neg_ext.pkl")


In [ ]:
df_soft_neg_ext.head()

In [ ]:
#1. Create packed sequences for RNN processing
#packed_queries, packed_positives, packed_negatives = create_packed_batch(df_all_neg_ext)



In [ ]:
# # 2. Feed packed sequences to your RNN models
# query_outputs, query_hidden =your_query_rnn(packed_queries)
# pos_outputs, pos_hidden = your_document_rnn(packed_positives)
# neg_outputs, neg_hidden = your_document_rnn(packed_negatives)

In [5]:
torch.cuda.empty_cache()  # Clear CUDA cache if using GPU


run_hyperparameter_tuning(df=df_soft_neg_ext, output_dims=[100], batch_sizes=[512], gru_hidden_dims=[100, 200], 
                         num_layers=[1], dropouts=[0.1], learning_rates=[0.0005, 1e-3], 
                         epochs=10, log_wandb=True)

Data splits: Train=47822 | Validation=15941 | Test=15941


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.




--------------------------------------------------------------------------------
Training with: output_dim=100, batch_size=512, gru_hidden_dim=100, num_layers=1, dropout=0.1, lr=0.0005
--------------------------------------------------------------------------------


/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/10 (Val): 100%|██████████| 32/32 [00:05<00:00,  5.85it/s]


Epoch 1/10, Train Loss: 0.1745, Val Loss: 0.1494, LR: 0.000500
New best model saved with validation loss: 0.1494


...

Epoch 10/10 (Val): 100%|██████████| 32/32 [00:04<00:00,  7.65it/s]


Epoch 10/10, Train Loss: 0.0495, Val Loss: 0.0662, LR: 0.001000
New best model saved with validation loss: 0.0662
Final model saved at: checkpoints/final_gru_model_20250425-151105/final_gru_model_20250425-151105.pt


[W425 15:23:37.812899285 shape_type_inference.cpp:1995] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W425 15:23:37.817799832 shape_type_inference.cpp:1995] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
/root/MLX_Week2/mlx_env/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with GRU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
[W425 15:23:37.842232926 shape_type_inference.cpp:1995] Warning: The shape in

Torch IR graph at exception: graph(%embeddings.1 : Float(1, 26, 100, strides=[2600, 100, 1], requires_grad=0, device=cuda:0),
      %1 : Long(1, strides=[1], requires_grad=0, device=cuda:0),
      %embeddings : Float(1, 201, 100, strides=[20100, 100, 1], requires_grad=0, device=cuda:0),
      %3 : Long(1, strides=[1], requires_grad=0, device=cuda:0),
      %query_encoder.gru.weight_ih_l0 : Float(300, 100, strides=[100, 1], requires_grad=1, device=cuda:0),
      %query_encoder.gru.weight_hh_l0 : Float(300, 100, strides=[100, 1], requires_grad=1, device=cuda:0),
      %query_encoder.gru.bias_ih_l0 : Float(300, strides=[1], requires_grad=1, device=cuda:0),
      %query_encoder.gru.bias_hh_l0 : Float(300, strides=[1], requires_grad=1, device=cuda:0),
      %query_encoder.gru.weight_ih_l0_reverse : Float(300, 100, strides=[100, 1], requires_grad=1, device=cuda:0),
      %query_encoder.gru.weight_hh_l0_reverse : Float(300, 100, strides=[100, 1], requires_grad=1, device=cuda:0),
      %query_

RuntimeError: vector::_M_range_check: __n (which is 0) >= this->size() (which is 0)

### Twin Tower Network

In [ ]:
model_path = os.path.join("checkpoints", "final_gru_model_20250424-152045", "final_gru_model_20250424-152045.pt")
print(f"Loading model from: {model_path}")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance
model = GRUTwinTowerModel(embedding_dim=100, gru_hidden_dim=100, output_dim=100, 
                         num_layers=1, dropout=0.1)

# Load the checkpoint and extract the model state dict
checkpoint = torch.load(model_path, map_location=device)
# The error shows the state_dict is nested under "model_state_dict"
model.load_state_dict(checkpoint["model_state_dict"])

model.to(device).eval()

print("Model loaded successfully!")

In [ ]:
# Loading the pre-trained model from WandB
#run = wandb.init()
# The correct artifact path format
api = wandb.Api()
artifact = api.artifact("nnamdi-odozi-ave-actuaries/gru-twin-tower-model/final_gru_model_20250424-174424:v0")

#https://wandb.ai/nnamdi-odozi-ave-actuaries/gru-twin-tower-model/artifacts/model/final_gru_model_20250424-174424/v0/files/final_gru_model_20250424-174424.pt
artifact_dir = artifact.download()

# Find the model file
import os
model_files = [f for f in os.listdir(artifact_dir) if f.endswith('.pt') or f.endswith('.pth')]
if not model_files:
    raise FileNotFoundError(f"No model files found in {artifact_dir}")

model_path = os.path.join(artifact_dir, model_files[0])
print(f"Found model at: {model_path}")

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(model_path, map_location=device)

# Create model with correct dimensions
model2 = GRUTwinTowerModel(
    embedding_dim=100, 
    gru_hidden_dim=100,  # Use 100 as seen in your model print
    output_dim=100,
    num_layers=1,
    dropout=0.1
)

# Load state dict (handle both formats)
if "model_state_dict" in checkpoint:
    model2.load_state_dict(checkpoint["model_state_dict"])
else:
    model2.load_state_dict(checkpoint)

model2 = model.to(device)
model2.eval()
print("Model loaded successfully!")

In [ ]:
print(model)

In [ ]:
# 2. Test with consecutive rows - just use a slice directly
df_slice = df_soft_neg_ext[0:3]  # Use any 3 consecutive rows

# Process dataframe slice
with torch.no_grad():
    # Move everything to device
    query_embs = torch.stack(df_slice['query_emb'].tolist()).to(device)
    query_lens = torch.tensor(df_slice['query_length'].tolist()).to(device)
    pos_embs = torch.stack(df_slice['pos_emb'].tolist()).to(device)
    pos_lens = torch.tensor(df_slice['pos_length'].tolist()).to(device)
    
    # Get encodings for all rows at once
    query_vecs, doc_vecs = model(query_embs, query_lens, pos_embs, pos_lens)
    
    # Calculate similarities
    sims = torch.nn.functional.cosine_similarity(query_vecs, doc_vecs, dim=1)
    
print("Similarities:", sims.cpu().numpy())

In [ ]:
df_slice

In [ ]:
# 2. Test with consecutive rows - just use a slice directly
df_slice = df_soft_neg_ext[0:512]  # Doing more rows

# Process dataframe slice
with torch.no_grad():
    # Move everything to device
    query_embs = torch.stack(df_slice['query_emb'].tolist()).to(device)
    query_lens = torch.tensor(df_slice['query_length'].tolist()).to(device)
    pos_embs = torch.stack(df_slice['pos_emb'].tolist()).to(device)
    pos_lens = torch.tensor(df_slice['pos_length'].tolist()).to(device)
    
    # Get encodings for all rows at once
    query_vecs, doc_vecs = model(query_embs, query_lens, pos_embs, pos_lens)
    
    # Calculate similarities
    sims = torch.nn.functional.cosine_similarity(query_vecs, doc_vecs, dim=1)
    
print("Similarities:", sims.cpu().numpy())
sims.cpu().numpy().mean()

In [ ]:
print(model)

In [ ]:
#with a random sentence:
query_test = "This is RBA"
doc_test = "This is RBA"
q_l = len(query_test.split())
d_l = len(doc_test.split())
 

In [ ]:
query_emb, q_l = text_to_embeddings(query_test, word_to_idx, embeddings, is_query=True)
doc_emb, d_l = text_to_embeddings(doc_test, word_to_idx, embeddings, is_query=False)
print(query_emb.shape)
print(doc_emb)
print(q_l, d_l)

In [ ]:
q = query_emb.mean(dim=0)
d = doc_emb.mean(dim=0)

In [ ]:
# Calculate similarities
sims = torch.nn.functional.cosine_similarity(q, d, dim=0)
    
print("Similarities:", sims.cpu().numpy())

In [ ]:
# 1. Make sure tensors are on the right device
device = next(model.parameters()).device
query_emb = query_emb.to(device)  # Shape should be [seq_length, embedding_dim]
q_l = torch.tensor([q_l], device=device)  # Single value for sequence length

# 2. Add batch dimension for model processing
query_emb = query_emb.unsqueeze(0)  # Shape becomes [1, seq_length, embedding_dim]


In [ ]:
# 3. Query-only inference using just dataframe columns 
#query_row = df_soft_neg_ext[0]  # Use any row
#test_query_emb = query_row['query_emb'].unsqueeze(0).to(device)
#test_query_len = torch.tensor([query_row['query_length']]).to(device)

# Just run through query encoder and tower
with torch.no_grad():
    query_encoded = model.query_encoder(query_emb, q_l)
    query_vector = model.query_tower(query_encoded)
    query_vector = torch.nn.functional.normalize(query_vector, p=2, dim=1) #I don't think this is needed, but let's keep it for now

print("Query vector shape:", query_vector.shape)
print("Values:", query_vector[0, :5].cpu().numpy())

In [ ]:
def evaluate_model(qryTower, docTower, dataloader, device):
    qryTower.eval()
    docTower.eval()

    total = 0
    correct = 0

    for batch in dataloader:
        # Get embeddings from batch
        query_emb = batch['query']
        pos_emb = batch['positive']
        neg_emb = batch['negative']
        
        # Forward pass through towers
        query_encoded = qryTower(query_emb)
        pos_encoded = docTower(pos_emb)
        neg_encoded = docTower(neg_emb)
        
        # Calculate similarities
        pos_sim = torch.nn.functional.cosine_similarity(query_encoded, pos_encoded)
        neg_sim = torch.nn.functional.cosine_similarity(query_encoded, neg_encoded)

        correct += (pos_sim > neg_sim).sum().item()
        total += batch['query'].size(0)

    acc = correct / total
    print(f"Eval Accuracy (query closer to pos than neg): {acc:.4f}")
    return acc



total_loss = 0
    

In [ ]:
#print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(dataloader):.4f}")
evaluate_model(final_qry_tower, final_doc_tower, dataloader, device)

In [ ]:
query_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)
pos_emb = text_to_embeddings("What is RBA", word_to_idx, embeddings)

# Ensure tensors have at least two dimensions before applying mean
if query_emb.dim() == 1:
	query_emb = query_emb.unsqueeze(0)
if pos_emb.dim() == 1:
	pos_emb = pos_emb.unsqueeze(0)

query_emb = query_emb.mean(dim=0)
pos_emb = pos_emb.mean(dim=0)

print(torch.nn.functional.cosine_similarity(query_emb, pos_emb, dim=0))

